# Analisando o bot

Este jupyter-notebook vai auxiliar na análise de detecção de intenções da Taís.

## Instalação

### Configurando jupyter

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Listando versões

In [2]:
import rasa_nlu

print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.14.1


### Treinando o modelo do Rasa NLU

Na celula abaixo todas as intents na pasta `../data/intents/` serão utilizadas para gerar a matrix de confuzão para de avaliação do bot.

Ela irá treinar o modelo e executar a avaliação que pode ser verificada na saída da célula abaixo.

In [4]:
import os

from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer
from rasa_nlu import config


intents_directory = '../../bot/data/intents/'

intents = {}

for intent_file in os.listdir(intents_directory):
    intent_file_path = os.path.join(intents_directory, intent_file)

    intents[intent_file] = {}

    intent_list = []
    intent_name = None

    with open(intent_file_path) as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()

            if line.startswith('##'):
                if intent_name is not None:
                    intents[intent_file][intent_name] = intent_list
                intent_name = line.replace('## intent:', '') 
                intent_list = []

            elif line.startswith('- '):
                intent_list.append(line.replace('- ', ''))


# loading the nlu training samples
training_data = load_data(intents_directory)

# trainer to educate our pipeline
trainer = Trainer(config.load("../../bot/nlu_config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/civel.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 96 (12 distinct intents)
	- Found intents: 'menu_lembrete', 'civel_defesa_digamais', 'civel_defesa', 'civel', 'civel_propositura_pessoajuridica', 'civel_propositura', 'civel_propositura_pessoafisica_localmoradia', 'civel_defesa_localprocesso', 'menu_lembrete_podepagaradvogado', 'menu_lembrete_naopodepagaradvogado', 'civel_propositura_pessoafisica_cejusc', 'civel_propositura_pessoafisica'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/actions.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 24 (2 distinct intents)
	- Found intents: 'action_confirma_agendamento', 'cejusc'
	- entity examples: 19 (1 distinct entities)
	- found entities: 'local'

INFO:rasa_nlu.training_

In [5]:
from rasa_nlu.evaluate import run_evaluation
run_evaluation('../../bot/data/intents/', model_directory)

INFO:tensorflow:Restoring parameters from /work/notebooks/intents/./models/nlu/default/current/intent_classifier_tensorflow_embedding.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/civel.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 96 (12 distinct intents)
	- Found intents: 'menu_lembrete', 'civel_defesa_digamais', 'civel_defesa', 'civel', 'civel_propositura_pessoajuridica', 'civel_propositura', 'civel_propositura_pessoafisica_localmoradia', 'civel_defesa_localprocesso', 'menu_lembrete_podepagaradvogado', 'menu_lembrete_naopodepagaradvogado', 'civel_propositura_pessoafisica_cejusc', 'civel_propositura_pessoafisica'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.training_data.loading:Training data format of ../../bot/data/intents/actions.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 24 (2 distinct intents)
	- Found intent

{'intent_evaluation': {'predictions': [{'text': 'cível',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9591062664985657},
   {'text': 'civel',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9470434188842773},
   {'text': 'Cível e Família',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9674227237701416},
   {'text': 'civel e familia',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9504618644714355},
   {'text': 'quero propor uma ação',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9659785032272339},
   {'text': 'quero processar alguem',
    'intent': 'civel',
    'predicted': 'civel_propositura',
    'confidence': 0.7798852920532227},
   {'text': 'estou sendo processado',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.9326409101486206},
   {'text': 'fui intimado a comparecer',
    'intent': 'civel',
    'predicted': 'civel',
    'confidence': 0.946544229

### Lista de Problemas

O arquivo `erros,json` mostra os erros encontrados após executar o comando `rasa_nlu.evaluate.run_evaluation`.

Normalmente os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.

Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [6]:
%cat errors.json

[
    {
        "text": "quero processar alguem",
        "intent": "civel",
        "intent_prediction": {
            "name": "civel_propositura",
            "confidence": 0.7798852920532227
        }
    },
    {
        "text": "como vai",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.7311261296272278
        }
    },
    {
        "text": "tudo bom",
        "intent": "tudo_bem",
        "intent_prediction": {
            "name": "cumprimentar",
            "confidence": 0.7270336747169495
        }
    },
    {
        "text": "bom dia",
        "intent": "tudo_bem",
        "intent_prediction": {
            "name": "cumprimentar",
            "confidence": 0.7537955641746521
        }
    }
]

### Avaliação do modelo de NLU treinado

Caso queira testar mensagens específicas mande a mensagem na célula seguinte:

In [8]:
pprint(interpreter.parse('oi'))

{
  "intent": {
    "name": "cumprimentar",
    "confidence": 0.8681807518005371
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "cumprimentar",
      "confidence": 0.8681807518005371
    },
    {
      "name": "elogios",
      "confidence": 0.30231115221977234
    },
    {
      "name": "civel_defesa",
      "confidence": 0.1436154842376709
    },
    {
      "name": "negar",
      "confidence": 0.11643615365028381
    },
    {
      "name": "afirmar",
      "confidence": 0.10478226095438004
    },
    {
      "name": "menu_lembrete_naopodepagaradvogado",
      "confidence": 0.0919896811246872
    },
    {
      "name": "despedir",
      "confidence": 0.08706281334161758
    },
    {
      "name": "andamento_processo",
      "confidence": 0.07254625856876373
    },
    {
      "name": "conhecimento",
      "confidence": 0.056591566652059555
    },
    {
      "name": "civel",
      "confidence": 0.05012869834899902
    }
  ],
  "text": "oi"
}
